# Комп'ютерний практикум №2
## Сидорський Володимир Сергійович (КА-21ф)
## Варіант 5

# Завдання

![Task_main](images/task_1.png)


![Task_input](images/task_2.png)

# Програма

In [1]:
import numpy as np
import pandas as pd

In [2]:
def rgmm(D):
    w = np.prod(D, axis=1) ** (1/D.shape[0])
    return w / w.sum(), {"or_w": w}

def compute_gci(D, or_w):
    n = D.shape[0]
    gci = 0
    # Sum only elements where i < j
    for i in range(n):
        for j in range(i + 1, n):
            gci += (np.log(D[i][j]) - np.log(or_w[i] / or_w[j])) ** 2
            
    gci *= (2 / (n -1) * (n-2))

    # Matrix dims: 0, 1, 2, 3, 4, 5
    THRESHS = [0, 0, 0, 0.1573, 0.3526, 0.370]
    thresh = THRESHS[n]

    if gci > thresh:
        print(f"GCI(={gci:.3f}) > {thresh} - МПП неузгоджена")
    elif gci > 0:
        print(f"GCI(={gci:.3f}) <= {thresh} - МПП допустимо узгоджена")
    else:
        print(f"GCI = 0.0 - МПП узгоджена")

def make_pretty_interval(interval):
    return f"[{round(interval[0], 4)}, {round(interval[1], 4)}]"

def compute_one_stability_index(interval):
    return min(1 / interval[0], interval[1])

def compute_stable_intervals_for_best_alt(input_mpp, input_weights, sort_weights=True, alternative_names=None):
    D = input_mpp.copy()
    weights = input_weights.copy()
    if sort_weights:
        sorted_ids = np.argsort(-weights)
        D = D[np.ix_(sorted_ids, sorted_ids)]
        weights = weights[sorted_ids]

    intervals_low, intervals_high = np.full_like(D, np.nan), np.full_like(D, np.nan) 
    n = D.shape[0]
    
    d_1_j_low = np.maximum(
        D[0, 1:] * (weights[1:] / weights[0]) ** (n/2),
        np.max(D[0, 1:, np.newaxis] * ((weights[1:]/weights[0])**n), axis=1)
    ) 
    intervals_low[0,1:] = d_1_j_low
    intervals_high[0,1:] = 9

    for k in range(1, n):
        for j in range(1, n):
            if j > k:
                intervals_low[k,j] = D[k,j] * ((weights[j] / weights[0]) ** n)
                intervals_high[k,j] = min(D[k,j] * ((weights[0] / weights[k]) ** n), 9)

    if sort_weights:
        sorted_back_ids = np.argsort(sorted_ids)
        intervals_low = intervals_low[np.ix_(sorted_back_ids, sorted_back_ids)]
        intervals_high = intervals_high[np.ix_(sorted_back_ids, sorted_back_ids)]
    
    all_intervals = np.stack([intervals_low, intervals_high], axis=-1)

    intervals_table = [
        [make_pretty_interval(all_intervals[i,j]) if not np.isnan(all_intervals[i,j]).all() else "-" for j in range(all_intervals.shape[1])]
        for i in range(all_intervals.shape[0])
    ]
    intervals_table = pd.DataFrame(intervals_table)
    if alternative_names is not None:
        intervals_table.columns = alternative_names
        intervals_table.index = alternative_names
    
    return all_intervals, intervals_table

def compute_stable_index_for_best_alt(stable_intervals, alternative_names=None):
    index_table = [
        [compute_one_stability_index(stable_intervals[i,j]) if not np.isnan(stable_intervals[i,j]).all() else "-" for j in range(stable_intervals.shape[1])]
        for i in range(stable_intervals.shape[0])
    ]
    index_table = pd.DataFrame(index_table)
    if alternative_names is not None:
        index_table.columns = alternative_names
        index_table.index = alternative_names
    return index_table

# Виклик програми

Вхідна матриця `A`

In [3]:
A = np.array([
    [1, 2, 3, 5, 7],
    [1/2, 1, 2, 2, 4],
    [1/3, 1/2, 1, 1, 2],
    [1/5, 1/2, 1, 1, 9],
    [1/7, 1/4, 1/2, 1/9, 1],
])
A

array([[1.        , 2.        , 3.        , 5.        , 7.        ],
       [0.5       , 1.        , 2.        , 2.        , 4.        ],
       [0.33333333, 0.5       , 1.        , 1.        , 2.        ],
       [0.2       , 0.5       , 1.        , 1.        , 9.        ],
       [0.14285714, 0.25      , 0.5       , 0.11111111, 1.        ]])

Знайдемо ваги альтернатив методом `RGMM` та дослідимо узгодженість за допомгою індексу `GCI`

In [4]:
alt_weights, add_output = rgmm(A)
alt_weights

array([0.44830326, 0.23320939, 0.12351047, 0.15065257, 0.0443243 ])

In [5]:
compute_gci(A, **add_output)

GCI(=2.527) > 0.37 - МПП неузгоджена


Відсортуємо альтернативи за спаданням важливості

In [6]:
sorted_idxs = np.argsort(-alt_weights)
sorted_alt_weights = alt_weights[sorted_idxs]
print("Альтернатива (вага): ", [f"a{idx+1}({alt_weights[idx]})" for idx in sorted_idxs])

Альтернатива (вага):  ['a1(0.4483032609422627)', 'a2(0.23320939182218123)', 'a4(0.1506525686498272)', 'a3(0.12351047388727412)', 'a5(0.044324304698454706)']


Відсортуємо відповідним чином матрицю `A` для спрощення подальших обчислень

In [7]:
sorted_A = A[np.ix_(sorted_idxs, sorted_idxs)]
sorted_A

array([[1.        , 2.        , 5.        , 3.        , 7.        ],
       [0.5       , 1.        , 2.        , 2.        , 4.        ],
       [0.2       , 0.5       , 1.        , 1.        , 9.        ],
       [0.33333333, 0.5       , 1.        , 1.        , 2.        ],
       [0.14285714, 0.25      , 0.11111111, 0.5       , 1.        ]])

Знайдемо інтервали стікйості `RSInt` локального ранжування альернатив для того щоб залишити домінуючу альернативу незмінною

In [8]:
numpy_intervals, intervals_table = compute_stable_intervals_for_best_alt(
    sorted_A, sorted_alt_weights, 
    sort_weights=False,
    alternative_names=[f"a{idx+1}" for idx in sorted_idxs]
)

In [9]:
intervals_table

,a1,a2,a4,a3,a5
a1,-,"[0.3904, 9.0]","[0.3273, 9.0]","[0.1195, 9.0]","[0.2667, 9.0]"
a2,-,-,"[0.0086, 9.0]","[0.0032, 9.0]","[0.0, 9.0]"
a4,-,-,-,"[0.0016, 9.0]","[0.0001, 9.0]"
a3,-,-,-,-,"[0.0, 9.0]"
a5,-,-,-,-,-


Знайдемо індекс стікйості `RSInd` локального ранжування альернатив для того щоб залишити домінуючу альернативу незмінною

In [10]:
index_table = compute_stable_index_for_best_alt(
    numpy_intervals,
    alternative_names=[f"a{idx+1}" for idx in sorted_idxs]
)

In [11]:
index_table

,a1,a2,a4,a3,a5
a1,-,2.561738,3.05505,8.3666,3.75
a2,-,-,9.0,9.0,9.0
a4,-,-,-,9.0,9.0
a3,-,-,-,-,9.0
a5,-,-,-,-,-


# Висновки

В даній роботі було знайдено інтервали та індекси стійкості щодо збереження ранжування альтернатив, а саме для збереження незмінної найкращої альтернативи. Для запропонованої МПП було знайдені ваги за допомогою методу `RGMM` , а потім знайдені інтервали та індекси стійкості згідно з формул з теоретичної частини. Всі використані методи було запрограмовано, текст програми наведено в звіті. 